In [411]:

import numpy as np
import pandas as pd
import operator
from matplotlib import pyplot as plt
%matplotlib inline

from random import shuffle

In [412]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")

In [413]:
test_array= test_data.to_numpy()
train_array=train_data.to_numpy()

In [449]:
class SVM():
    def __init__(self,lambd,nbr_features):
        self.lambd = lambd
        self.nbr_features = nbr_features# Numbre of features plus bias
        self.weights = [0.0] * self.nbr_features # Weights vector initialisation
        self.alpha = 0
        
    def data_processing(self, _array): #seperates features from target/sets target in {-1;1}/Adds a bias column to features
        _target = [0] * _array.shape[0]
        bias = np.ones((train_array.shape[0],1),float)
        _features = train_array[:,:train_array.shape[1]-1]
        _features = np.append(_features,bias,axis= 1)
        _features = _features.astype(float)
        for e in range(_array.shape[0]):
            if _array[e,_array.shape[1]-1] == 1:
                _target[e] = 1.
            else:
                _target[e] = -1.     
        return (_features, _target)
   
    def hinge_loss(self,target,y):
        return max(0, 1 - target*y)
    
    def sign(self, x):
        return -1. if x <= 0 else 1.
    
    def predict(self,_array): # predicts the class of the input
        result=0
        for i in range(len(_array)):
            result= result + self.weights[i] * _array[i]   
        return result
    
    def train(self,features,target):
        for i in range(features.shape[0]):
            self.alpha=(1./(self.lambd*(i+1.)))
            if target[i]*self.predict(features[i,:]) <1:
                for j in range(len(features[i,:])):
                    self.weights[j] =  self.weights[j] + self.alpha *( (target[i]*features[i,j]) + (-2 * (self.lambd)*self.weights[j]) )
            else:
                for j in range(len(features[i])):
                    self.weights[j] = self.weights[j] + self.alpha* (-2 * (self.lambd)*self.weights[j])

    
    def infer(self,features):
        predicted= []
        loss_pred =[]
        for i in range(features.shape[0]):
                predicted.append(self.sign(self.predict(features[i,:])))
        return predicted
    
    
    def performance(self,_predicted,_target):
        t_p = 0 
        t_n = 0
        f_p = 0
        f_n= 0
        t_p_n_array = [sum(x) for x in zip(_predicted, _target)] 
        f_p_n_array = list(map(operator.sub, _predicted, _target))
        
        for e in t_p_n_array:
                if e > 0 :
                    t_p = t_p + 1
                elif e <0 :
                    t_n = t_n + 1

        for e in f_p_n_array:
            if e >0 :
                f_p = f_p + 1
            elif e <0 :
                f_n = f_n + 1
                
        accuracy = (t_p + t_n)/(t_p + t_n + f_p + f_n)
        print("Accuracy =",accuracy*100, "%")
        loss = 0
        for i in range(len(_target)):
            loss = loss + self.hinge_loss(_target[i],_predicted[i])  
        print("loss =",loss/len(_target))
        
        
        
        
        
        

In [450]:
model= SVM(4,train_array.shape[1])
test_features , test_target= model.data_processing(test_array)
train_features , train_target= model.data_processing(train_array)

In [451]:
model.weights

[0.0, 0.0, 0.0, 0.0]

In [452]:
model.train(train_features,train_target)

In [453]:
model.weights

[-0.02871663117467558,
 -0.02509794197386528,
 -0.009765192275279022,
 -0.0049241953656784295]

In [454]:
predicted = model.infer(test_features)

In [455]:
model.performance(predicted,test_target)

Accuracy = 79.2568520448832 %
loss = 0.4148629591023361
